In [2]:
import pandas as pd
import numpy as np
import glob
import json

ModuleNotFoundError: No module named 'plotly'

# RDlite Examples

In [4]:
%load_ext autoreload
%autoreload 2

import RDlite.accessor as ac
from RDlite import mapping
from RDlite import all_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
all_features

dict_keys(['id', 'company', 'genre', 'keyword', 'language', 'rating', 'country', 'name', 'character', 'gender', 'adult', 'budget', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'title', 'video', 'vote_average', 'vote_count', 'collection'])

You may access the local data by specifying the feature names. If the peek keyword is set to true, the function only retrieves the first 50 rows to save time and memory. It is useful to get some sense of the data by browsing through examples.

In [10]:
ac.agg('genre', 'company', peek=True)

,genre,company
id,,
862,ANIMATION,PIXAR ANIMATION STUDIOS
8844,ADVENTURE,TRISTAR PICTURES
15602,ROMANCE,WARNER BROS.
31357,COMEDY,TWENTIETH CENTURY FOX FILM CORPORATION
11862,COMEDY,SANDOLLAR PRODUCTIONS
949,ACTION,REGENCY ENTERPRISES
11860,COMEDY,PARAMOUNT PICTURES
45325,ACTION,WALT DISNEY PICTURES
9091,ACTION,UNIVERSAL PICTURES


# Movie Clean Examples

In [19]:
import MovieClean.norm as norm

This package is useful for extracting data stored in json-style strings into either Series or DataFrame object.

In [27]:
example_json = {'name': 'MacFarlane', 'job': 'actor'}
example_frame = pd.DataFrame(
    {
        'id': range(10),
        'actors': [str([example_json]*5)]*10
    }
)
example_frame

,id,actors
0,0,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
1,1,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
2,2,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
3,3,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
4,4,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
5,5,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
6,6,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
7,7,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
8,8,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."
9,9,"[{'name': 'MacFarlane', 'job': 'actor'}, {'nam..."


Notice that in the example_frame, the actors column stores lists of json-style strings, which violates the first normal form of clean data.

Such data could be hard to handle with regex and the built-in json.parse function because the mixed useage of single and double quotes and the presence of special characters within the quotes, which violates the json format. 

In [29]:
norm.json2df(example_frame, 'actors', 'id').head()

,name,job,id
0,MacFarlane,actor,0
1,MacFarlane,actor,0
2,MacFarlane,actor,0
3,MacFarlane,actor,0
4,MacFarlane,actor,0


Note: Since there are multiple data entries within a cell, the total amount of data extracted might be bigger than expected, which is slow to process. Thus, it is recommended to use .json2series to extract a single key-value pair of interest.

In [31]:
norm.json2series(example_frame, 'actors', 'name', 'id').head()

id
0    MacFarlane
1    MacFarlane
2    MacFarlane
3    MacFarlane
4    MacFarlane
Name: name, dtype: object

Sometimes you may find the index of a dataframe is 'object' but you intend to make it int. 

In [36]:
example_frame = pd.DataFrame({
    'id': [str(num) for num in range(10)],
    'actors': ['Lois'] * 10
})
example_frame.set_index('id').index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object', name='id')

In [38]:
# Easy way to convert it
norm.id2int(example_frame).index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64', name='id')

For simple categorical feature, you can encode it.

In [39]:
cat = ['actor', 'director', 'screen_writer']
example_frame = pd.DataFrame({
    'id': range(10),
    'actors': [cat[num%3] for num in range(10)]
})
example_frame

,id,actors
0,0,actor
1,1,director
2,2,screen_writer
3,3,actor
4,4,director
5,5,screen_writer
6,6,actor
7,7,director
8,8,screen_writer
9,9,actor


In [41]:
# Encode
norm.cat_encode(example_frame, 'actors')

/workspaces/CSE314Final/MovieClean/norm.py:138: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, col_name] = data[col_name].apply(lambda x: mapping[x])


0            actor
1         director
2    screen_writer
Name: actors_map, dtype: object

In [42]:
example_frame

,id,actors
0,0,0
1,1,1
2,2,2
3,3,0
4,4,1
5,5,2
6,6,0
7,7,1
8,8,2
9,9,0
